In [2]:
import pandas as pd
import numpy as np

from google.cloud import bigquery


In [3]:
#Authentication

#This should work in AI Platform Notebooks
#client = bigquery.Client()

#Pass the Service Acount key if script is being run outside the GCP Project
client = bigquery.Client.from_service_account_json('<PATH to Service account key *.json file>')

In [4]:

#Retrieve Sample data from BQ
sql = """
SELECT * FROM `jsb-demos.yellow_taxi.yellow_taxi_fare_2m` 
#WHERE pickup_datetime < '2016-01-03'
ORDER BY pickup_datetime 
LIMIT 100000
"""
#Convert to DataFrame
train_data = client.query(sql).to_dataframe()

In [5]:
#Random Split data into two sets. 
#Dummy step. In actual deployment assign two data sets to be validated to df1 and df2
msk = np.random.rand(len(train_data)) < 0.5
df1 = train_data[msk]
df2 = train_data[~msk]

In [14]:
#Define function to compare two datasets and calculate deltas in key stats

def evaluate_drift(old_data, new_data):
    df1 = old_data
    df2 = new_data

    #Calculate key stats
    df1_sts = df1.describe()
    df2_sts = df2.describe()

    rows = df1_sts.index.values
    columns = df1_sts.columns.values
    
    #Append blank fields
    df1_sts = df1_sts.append(pd.Series(name='skew',dtype="float64"))
    df1_sts = df1_sts.append(pd.Series(name='kurt',dtype="float64"))
    df1_sts = df1_sts.append(pd.Series(name='percent_missing',dtype="float64"))
    
    df2_sts = df2_sts.append(pd.Series(name='skew',dtype="float64"))
    df2_sts = df2_sts.append(pd.Series(name='kurt',dtype="float64"))
    df2_sts = df2_sts.append(pd.Series(name='percent_missing',dtype="float64"))

    #Calculate skew, kurt, missing% for each field
    for col in columns:
        df1_sts[col]['skew'] = df1[col].skew()
        df1_sts[col]['kurt'] = df1[col].kurt()
        df1_sts[col]['percent_missing'] = df1[col].isnull().sum()/len(df1)*100
        
        df2_sts[col]['skew'] = df2[col].skew()
        df2_sts[col]['kurt'] = df2[col].kurt()
        df2_sts[col]['percent_missing'] = df2[col].isnull().sum()/len(df2)*100
        


    rows = df1_sts.index.values
    columns = df1_sts.columns.values
    drift_summary = pd.DataFrame(columns=columns, index=rows)

    #Calculate the delta(%) for each field between the two datasets
    for col in columns:
        for row in rows:
            if (df1_sts[col][row] != 0):
                drift_value = (df1_sts[col][row] - df2_sts[col][row])/df1_sts[col][row]
            elif (df2_sts[col][row] != 0):
                drift_value= (df1_sts[col][row] - df2_sts[col][row])/df2_sts[col][row]
            else:
                drift_value = 0


            drift_summary[col][row] = abs(round(drift_value, 2))

    return(df1_sts,df2_sts, drift_summary)


In [16]:
df1_sts, df2_sts, drift_summary = evaluate_drift(df1,df2)

drift_summary

,trip_distance,pickup_latitude,pickup_longitude,rate_code,dropoff_latitude,dropoff_longitude,fare_amount
count,0,0,0,0,0,0,0
mean,0,0,0,0,0,0,0
std,0.01,0.15,0.09,0.01,0.28,2.87,0.02
min,0,0.1,0.07,0,0.25,0.64,0.4
25%,0,0,0,0,0,0,0
50%,0,0,0,0,0,0,0
75%,0.01,0,0,0,0,0,0.02
max,0.07,0.04,0.01,0,0,0.01,0.5
skew,0.08,0.93,1.23,0,0.92,57.78,0.46
kurt,0.28,1,0.97,0.02,0.97,423.74,1.68


In [17]:
df1_sts

,trip_distance,pickup_latitude,pickup_longitude,rate_code,dropoff_latitude,dropoff_longitude,fare_amount
count,50001.000000,50001.000000,50001.000000,50001.000000,50001.000000,50001.000000,50001.000000
mean,5.955042,40.745318,-73.954695,1.097978,40.747262,-73.960669,20.890821
std,5.484222,0.044623,0.070149,0.453983,0.067995,0.057317,16.172480
min,0.000000,36.671406,-80.310768,1.000000,29.886585,-74.576019,2.500000
25%,1.700000,40.731152,-73.990883,1.000000,40.724243,-73.989792,8.000000
50%,4.520000,40.753963,-73.978523,1.000000,40.752419,-73.975021,18.000000
75%,9.100000,40.769779,-73.954170,1.000000,40.771946,-73.949791,29.000000
max,179.400000,42.994476,-71.537590,6.000000,41.386200,-71.537590,568.500000
skew,2.001222,-17.797204,-12.339851,6.311311,-82.766058,3.551149,3.397756
kurt,24.703248,1785.802278,1391.591366,45.944657,13068.112912,102.582808,57.201410


In [18]:
df2_sts

,trip_distance,pickup_latitude,pickup_longitude,rate_code,dropoff_latitude,dropoff_longitude,fare_amount
count,49999.000000,49999.000000,49999.000000,49999.000000,49999.000000,49999.000000,49999.000000
mean,5.944047,40.745553,-73.954584,1.098862,40.747335,-73.961569,20.914543
std,5.453507,0.037853,0.063914,0.459799,0.048638,0.222051,16.445804
min,0.000000,40.316799,-74.658905,1.000000,37.389381,-121.933258,1.500000
25%,1.700000,40.731730,-73.990753,1.000000,40.724045,-73.989761,8.000000
50%,4.500000,40.754009,-73.978523,1.000000,40.752354,-73.975174,18.000000
75%,9.030000,40.769812,-73.954254,1.000000,40.771801,-73.949783,28.500000
max,166.200000,41.127060,-71.918953,6.000000,41.317001,-71.945045,855.000000
skew,1.832415,-1.195564,2.834112,6.286728,-6.637567,-201.631802,4.975185
kurt,17.663627,3.207814,42.737035,45.224503,457.041458,43571.437105,153.433994
